# Calculate robusteness from pi control runs

In [1]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [2]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [3]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'anomaly ts/'+runs+'_'+loc+'_sst_anom.nc'
    fin2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp)) #/(y2-y1+1)) #freq.
        
    return nev  # sum(nev)

In [5]:
ny = 30
prc1 = 90
prc2 = 10
loc='goa'

pinev=list()
for i in range(int(500/ny)):
    nev = num_coevents(i*ny, (i+1)*ny-1, prc1 , prc2,loc=loc)
    pinev.append(nev)
pinev,np.mean(pinev) 

FileNotFoundError: [Errno 2] No such file or directory: b'/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/anomaly ts/picontrol_goa_sst_anom.nc'

In [ ]:
#model by model

In [ ]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

In [ ]:
#16 30-yr periods MMEs

In [ ]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

In [ ]:
#MME mean and spread

In [ ]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

In [ ]:
#cross model mean and spread

In [ ]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

In [ ]:
## Calculate thresholds for PI Control runs

In [ ]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'anomaly ts/picontrol_'+loc+'_sst_anom.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'anomaly ts/picontrol_sm_anom.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [ ]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, loc, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'anomaly ts/'+runs+'_'+loc+'_sst_anom.nc'
    fi2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev, np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [ ]:
def freq_difruns(loc,ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc, mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future 1
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq3, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    # future 2
    y2=2069
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq4, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    return freq0,freq1,freq2,freq3,freq4

In [ ]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [ ]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

funev=list()
hinev=list()
funev2=list()
hinev2=list()
test=list()
for i in range(int(502/ny)):
    pi,hi,hi2,fu,fu2 = freq_difruns(loc,ny,i*ny, (i+1)*ny-1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    funev2.append(fu2)
    hinev.append(hi)
    hinev2.append(hi2)


In [60]:
np.mean(funev),np.mean(funev2)

(0.6960227272727273, 1.0767045454545454)

In [61]:
np.mean(hinev),np.mean(hinev2)

(1.0170454545454546, 0.9176136363636364)

In [62]:
np.mean(test),np.mean(pinev)

(0.32386363636363635, 0.32386363636363635)

In [63]:
#model by model

In [64]:
fu_model_mean=np.mean(funev,axis=0)
fu_model_spread=np.std(funev,axis=0)
fu_model_mean

array([0.125 , 0.3125, 1.5   , 0.5625, 0.25  , 0.3125, 0.875 , 0.4375,
       1.0625, 1.0625, 0.375 , 0.9375, 1.3125, 1.1875, 0.25  , 1.0625,
       0.125 , 0.0625, 0.5625, 1.3125, 1.25  , 0.375 ])

In [65]:
np.mean(fu_model_mean),np.std(fu_model_mean)

(0.6960227272727273, 0.4523829953518049)

In [66]:
#mme mean and spread

In [67]:
fu_mme_mean=np.mean(funev,axis=1)
fu_mme_spread=np.std(funev,axis=1)
np.mean(fu_mme_spread)

0.9762132316532354

In [68]:
fu_mme_mean,np.mean(fu_mme_mean),np.std(fu_mme_mean)

(array([0.59090909, 0.63636364, 0.40909091, 0.68181818, 0.5       ,
        0.40909091, 0.81818182, 0.81818182, 1.09090909, 0.77272727,
        0.5       , 0.59090909, 0.95454545, 0.59090909, 1.40909091,
        0.36363636]),
 0.6960227272727273,
 0.2687776268713815)

In [69]:
#16 MMEs diff and spread

In [70]:
diff=list()
zip_obj=zip(fu_mme_mean,mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.09090909090909094,
 0.36363636363636365,
 0.04545454545454547,
 0.4545454545454545,
 0.0,
 0.0,
 0.6363636363636365,
 0.4545454545454546,
 0.8636363636363635,
 0.5454545454545454,
 0.3181818181818182,
 0.27272727272727276,
 0.5454545454545454,
 0.2272727272727273,
 1.1363636363636365,
 0.0]

In [71]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3721590909090909, 0.3166435013094183)

In [72]:
#model by model diff and spread

In [73]:
diff=list()
zip_obj=zip(fu_model_mean,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[-0.125,
 -0.125,
 1.25,
 0.125,
 0.0625,
 -0.125,
 0.75,
 0.3125,
 0.75,
 0.9375,
 -0.0625,
 0.375,
 0.9375,
 0.75,
 0.0625,
 0.5,
 -0.25,
 -0.1875,
 0.1875,
 0.875,
 1.0625,
 0.125]

In [74]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3721590909090909, 0.45558297783179147)